<a href="https://colab.research.google.com/github/rsbensaid/DFS/blob/master/spurs_team_intro_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Access & Storage Examples: Spurs

# Notebook Setup

In [0]:
#@title Authenticate
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [0]:
#@title Library Imports & Client Setup
from google.cloud import bigquery
import logging
from datetime import datetime
project = 'spurs-sp2018'
bq_client = bigquery.Client(project=project)
gcs_client = bigquery.Client(project=project)

# BigQuery Access

[BigQuery](https://console.cloud.google.com/bigquery?project=spurs-sp2018&authuser=0) is the relational database where all of your data will be stored

**Via Magic Commands**

Top 5 Teams By FG% From > 40

In [0]:
%%bigquery --project spurs-sp2018 ins\rt_df

SELECT season
, abbrev
, ROUND(AVG(IF(outcome, 1, 0)), 3) AS fg_perc 
FROM ss.shots AS s
LEFT JOIN ss.teams AS t
ON s.off_team_id = t.id
WHERE distance > 40
GROUP BY season
, abbrev
ORDER BY fg_perc DESC
LIMIT 5

Executing query with job ID: 5d4996c8-00de-4bf2-bf8e-135e67ac0428
Query executing: 0.37s

In [0]:
print(insert_df)

  season abbrev  fg_perc
0   2017    POR    0.300
1   2016    LAL    0.240
2   2013    LAL    0.214
3   2015    CLE    0.211
4   2017    LAC    0.200


**Via Python Client**


Check tables for duplicate values

In [0]:
tables = ['chances', 'possessions', 'shots', 'rebounds', 'turnovers'
          , 'free_throws', 'fouls', 'timeouts', 'violations', 'passes'
          , 'touches', 'dribbles', 'picks', 'isolations', 'handoffs', 'drives'
          , 'posts', 'off_ball_screens', 'fake_handoffs', 'closeouts'
          , 'chance_players', 'rotations', 'poss_matchups', 'speed_loads'
]

for t in tables:
  sql = """
    SELECT game_id
    , game_date
    , path
    FROM (
      SELECT game_id, id, COUNT(*) AS x FROM ss.{_t_} GROUP BY game_id, id
    ) AS y
    LEFT JOIN ss.games AS g
    ON y.game_id = g.id
    WHERE x > 1
    GROUP BY game_id
    , game_date
    , path
  """.format(_t_ = t)
  df = bq_client.query(sql).to_dataframe()
  if not df.empty:
    print(t + ':' + str(len(list(df['path']))))
    print(t + ':' + str(tuple(df['game_id'])))
  else:
    print(t + ': NO DUPLICATES')

chances: NO DUPLICATES
possessions:1
possessions:('2d5c731b-6aa1-4589-b3ba-8395f47b30ef',)
shots: NO DUPLICATES
rebounds: NO DUPLICATES
turnovers: NO DUPLICATES
free_throws: NO DUPLICATES
fouls: NO DUPLICATES
timeouts: NO DUPLICATES
violations: NO DUPLICATES
passes: NO DUPLICATES
touches: NO DUPLICATES
dribbles: NO DUPLICATES
picks: NO DUPLICATES
isolations: NO DUPLICATES
handoffs: NO DUPLICATES
drives: NO DUPLICATES
posts: NO DUPLICATES
off_ball_screens: NO DUPLICATES
fake_handoffs: NO DUPLICATES
closeouts: NO DUPLICATES
chance_players: NO DUPLICATES
rotations: NO DUPLICATES
poss_matchups:1
poss_matchups:('2d5c731b-6aa1-4589-b3ba-8395f47b30ef',)
speed_loads: NO DUPLICATES


# Accessing Raw Files

Raw files will be stored in [Google Cloud Storage](https://console.cloud.google.com/storage/browser?project=spurs-sp2018&authuser=0)